In [1]:
import cobra
import numpy as np
import pandas as pd
import sys

source_dir = '../src'
sys.path.append(source_dir)

from get_gene_transcript_dictionary import get_gene_transcript_dictionary
from get_reaction_transcript_dictionary import get_reaction_transcript_dictionary
from eflux2 import eflux2
from add_column_to_13c_flux_df import add_column_to_13c_flux_df


### Load genome scale model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr_2.json")
model

Name,model
Memory address,1180fbf70
Number of metabolites,1121
Number of reactions,1348
Number of genes,648
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Load transcriptomics data

In [3]:
# load this file data/transcriptomics/fpkm_counts.csv
transcriptomics_df = pd.read_csv("../data/transcriptomics/fpkm_counts.csv")
transcriptomics_df

,JGI_id,YALI0_id,glucose_1,glucose_2,glucose_3,glycerol_1,glycerol_2,glycerol_3,oleic_acid_1,oleic_acid_2,oleic_acid_3
0,jgi.p|Yarli1|64471,YALI0A00110g,4.88,2.64,3.23,1.21,1.11,3.61,6.77,13.32,7.67
1,jgi.p|Yarli1|64472,YALI0A00132g,1896.43,1845.77,1682.56,1234.99,1189.24,1120.35,864.64,1056.63,928.51
2,jgi.p|Yarli1|64473,YALI0A00154g,31.34,29.21,21.92,215.23,137.85,182.81,1867.59,2153.21,1958.68
3,jgi.p|Yarli1|64474,YALI0A00176g,3.67,3.85,3.15,3.07,1.74,2.09,4.13,2.43,4.34
4,jgi.p|Yarli1|64475,YALI0A00198g,0.00,0.17,0.00,0.00,0.50,1.29,1.55,0.81,1.75
...,...,...,...,...,...,...,...,...,...,...,...
6442,jgi.p|Yarli1|70913,YALI0F32043g,20.63,17.60,17.12,13.94,13.84,11.94,10.76,12.38,10.37
6443,jgi.p|Yarli1|70914,YALI0F32065g,192.58,151.54,270.76,206.89,262.66,280.82,310.55,384.22,313.24
6444,jgi.p|Yarli1|70915,YALI0F32131g,0.12,0.66,0.14,0.34,0.29,0.64,1.01,0.58,1.33
6445,jgi.p|Yarli1|70916,YALI0F32153g,26.16,27.73,28.77,39.22,36.95,39.18,26.98,22.05,23.58


### Make gene -> transcript dictionaries for each condition

In [4]:
glucose_1_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glucose_1')
glucose_2_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glucose_2')
glucose_3_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glucose_3')

glycerol_1_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glycerol_1')
glycerol_2_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glycerol_2')
glycerol_3_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'glycerol_3')

oleic_acid_1_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'oleic_acid_1')
oleic_acid_2_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'oleic_acid_2')
oleic_acid_3_gene_transcripts = get_gene_transcript_dictionary(transcriptomics_df, 'oleic_acid_3')

glucose_1_gene_transcripts


{'': inf,
 'YALI0A00110g': 4.88,
 'YALI0A00132g': 1896.43,
 'YALI0A00154g': 31.34,
 'YALI0A00176g': 3.67,
 'YALI0A00198g': 0.0,
 'YALI0A00212g': 9.15,
 'YALI0A00264g': 489.68,
 'YALI0A00286g': 244.82,
 'YALI0A00330g': 32.58,
 'YALI0A00352g': 1691.33,
 'YALI0A00369g': 0.81,
 'YALI0A00374g': 2.64,
 'YALI0A00396g': 33.59,
 'YALI0A00418g': 7.0,
 'YALI0A00440g': 137.53,
 'YALI0A00506g': 44.11,
 'YALI0A00528g': 0.0,
 'YALI0A00550g': 4.56,
 'YALI0A00572g': 0.68,
 'YALI0A00594g': 476.9,
 'YALI0A00616g': 40.21,
 'YALI0A00638g': 57.82,
 'YALI0A00660g': 18.78,
 'YALI0A00682g': 0.0,
 'YALI0A00759g': 30.68,
 'YALI0A00781g': 38.97,
 'YALI0A00803g': 145.71,
 'YALI0A00825g': 11.83,
 'YALI0A00847g': 23.73,
 'YALI0A00869g': 0.22,
 'YALI0A00891g': 148.29,
 'YALI0A00913g': 52.82,
 'YALI0A00935g': 2.52,
 'YALI0A00957g': 16.08,
 'YALI0A00979g': 31.73,
 'YALI0A01001g': 43.35,
 'YALI0A01023g': 116.43,
 'YALI0A01045g': 70.65,
 'YALI0A01067g': 142.27,
 'YALI0A01089g': 10.03,
 'YALI0A01111g': 39.55,
 'YALI0A0113

### Make reaction -> transcript dictionaries for each condition

In [5]:
# glucose_1_reaction_transcripts = get_reaction_transcript_dictionary(model, glucose_1_gene_transcripts)
# glucose_2_reaction_transcripts = get_reaction_transcript_dictionary(model, glucose_2_gene_transcripts)
# glucose_3_reaction_transcripts = get_reaction_transcript_dictionary(model, glucose_3_gene_transcripts)

# glucose_1_reaction_transcripts

### Run E-Flux2 for Glucose 1

In [6]:
glucose_medium = {
    'EX_glc(e)': np.inf,
    'EX_h2o(e)': 1000.0,
    'EX_h(e)': 1000.0,
    'EX_nh4(e)': 1000.0,
    'EX_o2(e)': 1000.0,
    'EX_pi(e)': 1000.0,
    'EX_so4(e)': 1000.0,
}

glucose_1_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=glucose_1_gene_transcripts, 
    medium=glucose_medium, 
    carbon_uptake_reaction='EX_glc(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
glucose_1_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glucose_1_gsm_eflux2.csv', index=False)

display(glucose_1_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-666.959559
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,366.566172
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],365.261313
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],365.261313
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-320.522192
421,EX_h2o(e),H2O exchange,h2o[e] <=>,320.522192
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,316.638002
639,NADH2_u6cm,NADH dehydrogenase cytosolicmitochondrial,h[c] + nadh[c] + q6[m] --> nad[c] + q6h2[m],281.289436
291,GLUDy,glutamate dehydrogenase NADP,glu_L[c] + h2o[c] + nadp[c] <=> akg[c] + h[c] ...,-187.019702
391,EX_co2(e),CO2 exchange,co2[e] -->,184.301252


### Run E-Flux2 for Glucose 2

In [7]:
glucose_2_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=glucose_2_gene_transcripts, 
    medium=glucose_medium, 
    carbon_uptake_reaction='EX_glc(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
glucose_2_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glucose_2_gsm_eflux2.csv', index=False)

display(glucose_2_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-667.557678
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,366.662025
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],365.378556
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],365.378556
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-316.963941
421,EX_h2o(e),H2O exchange,h2o[e] <=>,316.963941
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,316.873988
639,NADH2_u6cm,NADH dehydrogenase cytosolicmitochondrial,h[c] + nadh[c] + q6[m] --> nad[c] + q6h2[m],282.104852
957,CO2t,CO2 transporter via diffusion,co2[e] <=> co2[c],-185.010774
391,EX_co2(e),CO2 exchange,co2[e] -->,185.010774


### Run E-Flux2 for Glucose 3

In [8]:
glucose_3_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=glucose_1_gene_transcripts, 
    medium=glucose_medium, 
    carbon_uptake_reaction='EX_glc(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
glucose_3_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glucose_3_gsm_eflux2.csv', index=False)

display(glucose_3_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-666.959559
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,366.566172
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],365.261313
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],365.261313
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-320.522192
421,EX_h2o(e),H2O exchange,h2o[e] <=>,320.522192
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,316.638002
639,NADH2_u6cm,NADH dehydrogenase cytosolicmitochondrial,h[c] + nadh[c] + q6[m] --> nad[c] + q6h2[m],281.289436
957,CO2t,CO2 transporter via diffusion,co2[e] <=> co2[c],-184.301252
391,EX_co2(e),CO2 exchange,co2[e] -->,184.301252


### Glucose Central Metabolism E-Flux2

In [9]:
# load glucose 13C MFA data
glucose_13c_mfa_df = pd.read_csv('../data/13c_mfa/glucose_13c_mfa.csv')

# only keep the reactions that are mapped to the GSM
glucose_13c_mfa_df.dropna(subset = ["reaction_ids"], inplace=True)

# add the GSM flux predictions to the 13C-MFA dataframe
glucose_central_eflux2_df = add_column_to_13c_flux_df(glucose_13c_mfa_df, glucose_1_gsm_eflux2_df, 'Glucose 1 Flux')
glucose_central_eflux2_df = add_column_to_13c_flux_df(glucose_central_eflux2_df, glucose_2_gsm_eflux2_df, 'Glucose 2 Flux')
glucose_central_eflux2_df = add_column_to_13c_flux_df(glucose_central_eflux2_df, glucose_3_gsm_eflux2_df, 'Glucose 3 Flux')

# save the central fluxes as .csv
glucose_central_eflux2_df.to_csv('../results/eflux2_central_fluxes/glucose_central_eflux2.csv', index=False)

glucose_central_eflux2_df

,pathway,compartment,13C_mfa_reaction,reaction_ids,13C_mfa_flux,13C_mfa_flux_95%_lower_bound,13C_mfa_flux_95%_upper_bound,Forward Flux,Backward Flux,Stdev,Glucose 1 Flux,Glucose 2 Flux,Glucose 3 Flux
0,substrate_uptake,cytosol,Glucose(substrate) + ATP == G6P,GLCt1,100.0000,100.0000,100.0000,[],[],0.0000,1.000000e+02,100.000000,100.000000
1,substrate_uptake,cytosol,Glycerol(substrate) == GLYC,reverse_GLYCt,0.0000,0.0000,1.2414,[],[],0.3167,-7.248396e+00,-9.164436,-7.248396
2,substrate_uptake,cytosol,Acetate(substrate) == AC,ACt2r,0.0678,0.0000,0.7662,[],[],0.1955,0.000000e+00,0.000000,0.000000
3,substrate_uptake,cytosol,oleic_acid(substrate) == AC,OCDCEAt,0.0000,0.0000,0.0000,[],[],0.0000,0.000000e+00,0.000000,0.000000
4,emp,cytosol,G6P == F6P,PGI,72.6860,58.4274,78.3064,72.686,0,5.0712,-4.354580e+01,-44.923378,-43.545797
5,emp,cytosol,F6P + ATP == FBP,PFK or reverse_FBP,80.4599,71.9969,81.0889,149.0655,68.6056,2.3194,0.000000e+00,0.000000,0.000000
6,emp,cytosol,FBP == DHAP + GAP,FBA,80.4599,71.9969,81.0889,162.3111,81.8512,2.3194,0.000000e+00,0.000000,0.000000
7,emp,cytosol,DHAP == GAP,TPI,-19.5394,-25.9334,-18.2523,7.6658,27.2052,1.9595,2.307489e+01,21.057458,23.074888
8,emp,cytosol,GAP == G3P + ATP + NADH,GAPD,66.2012,54.6225,66.7364,380.1085,313.9073,3.0903,9.697373e+01,95.384146,96.973729
9,emp,cytosol,G3P == PEP,ENO,65.6276,53.6544,66.0194,1469.4,1403.8,3.1543,8.852677e+01,87.075647,88.526768


### Run E-Flux2 for Glycerol 1

In [10]:
glycerol_medium = {
    'EX_glyc(e)': np.inf,
    'EX_h2o(e)': 1000.0,
    'EX_h(e)': 1000.0,
    'EX_nh4(e)': 1000.0,
    'EX_o2(e)': 1000.0,
    'EX_pi(e)': 1000.0,
    'EX_so4(e)': 1000.0,
}

glycerol_1_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=glycerol_1_gene_transcripts, 
    medium=glycerol_medium, 
    carbon_uptake_reaction='EX_glyc(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
glycerol_1_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glycerol_1_gsm_eflux2.csv', index=False)

display(glycerol_1_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-432.201100
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,249.927329
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],249.651098
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],226.995975
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-220.541970
421,EX_h2o(e),H2O exchange,h2o[e] <=>,220.541970
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,206.615100
964,O2t,o2 transport diffusion,o2[e] <=> o2[c],145.185610
444,EX_o2(e),O2 exchange,o2[e] <=>,-145.185610
846,NADH2_u6m,NADH dehydrogenase mitochondrial,h[m] + nadh[m] + q6[m] --> nad[m] + q6h2[m],131.563311


### Run E-Flux2 for Glycerol 2

In [11]:
glycerol_2_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=glycerol_2_gene_transcripts, 
    medium=glycerol_medium, 
    carbon_uptake_reaction='EX_glyc(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
glycerol_2_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glycerol_2_gsm_eflux2.csv', index=False)

display(glycerol_2_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-430.419575
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,252.569690
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],244.710288
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],222.293524
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-221.317091
421,EX_h2o(e),H2O exchange,h2o[e] <=>,221.317091
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,207.386691
964,O2t,o2 transport diffusion,o2[e] <=> o2[c],146.458988
444,EX_o2(e),O2 exchange,o2[e] <=>,-146.458988
846,NADH2_u6m,NADH dehydrogenase mitochondrial,h[m] + nadh[m] + q6[m] --> nad[m] + q6h2[m],130.182383


### Run E-Flux2 for Glycerol 3

In [12]:
glycerol_3_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=glycerol_1_gene_transcripts, 
    medium=glycerol_medium, 
    carbon_uptake_reaction='EX_glyc(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
glycerol_3_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/glycerol_3_gsm_eflux2.csv', index=False)

display(glycerol_3_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-432.201100
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,249.927329
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],249.651098
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],226.995975
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-220.541970
421,EX_h2o(e),H2O exchange,h2o[e] <=>,220.541970
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,206.615100
964,O2t,o2 transport diffusion,o2[e] <=> o2[c],145.185610
444,EX_o2(e),O2 exchange,o2[e] <=>,-145.185610
846,NADH2_u6m,NADH dehydrogenase mitochondrial,h[m] + nadh[m] + q6[m] --> nad[m] + q6h2[m],131.563311


### Glycerol Central Metabolism E-Flux2

In [13]:
# load glycerol 13C MFA data
glycerol_13c_mfa_df = pd.read_csv('../data/13c_mfa/glycerol_13c_mfa.csv')

# only keep the reactions that are mapped to the GSM
glycerol_13c_mfa_df.dropna(subset = ["reaction_ids"], inplace=True)

# add the GSM flux predictions to the 13C-MFA dataframe
glycerol_central_eflux2_df = add_column_to_13c_flux_df(glycerol_13c_mfa_df, glycerol_1_gsm_eflux2_df, 'Glycerol 1 Flux')
glycerol_central_eflux2_df = add_column_to_13c_flux_df(glycerol_central_eflux2_df, glycerol_2_gsm_eflux2_df, 'Glycerol 2 Flux')
glycerol_central_eflux2_df = add_column_to_13c_flux_df(glycerol_central_eflux2_df, glycerol_3_gsm_eflux2_df, 'Glycerol 3 Flux')

# save the central fluxes as .csv
glycerol_central_eflux2_df.to_csv('../results/eflux2_central_fluxes/glycerol_central_eflux2.csv', index=False)

glycerol_central_eflux2_df

,pathway,compartment,13C_mfa_reaction,reaction_ids,13C_mfa_flux,13C_mfa_flux_95%_lower_bound,13C_mfa_flux_95%_upper_bound,Forward Flux,Backward Flux,Stdev,Glycerol 1 Flux,Glycerol 2 Flux,Glycerol 3 Flux
0,substrate_uptake,cytosol,Glucose(substrate) + ATP == G6P,GLCt1,0,NaN,NaN,[],[],NaN,0.000000,0.000000,0.000000
1,substrate_uptake,cytosol,Glycerol(substrate) == GLYC,reverse_GLYCt,100,NaN,NaN,[],[],NaN,100.000000,100.000000,100.000000
2,substrate_uptake,cytosol,Acetate(substrate) == AC,ACt2r,0,NaN,NaN,[],[],NaN,0.000000,0.000000,0.000000
3,substrate_uptake,cytosol,oleic_acid(substrate) == AC,OCDCEAt,0,NaN,NaN,[],[],NaN,0.000000,0.000000,0.000000
4,emp,cytosol,G6P == F6P,PGI,6,NaN,NaN,420.0208,426.1404,NaN,-2.283861,-2.295899,-2.283861
5,emp,cytosol,F6P + ATP == FBP,PFK or reverse_FBP,9,NaN,NaN,0,8.679,NaN,-32.316964,-32.931868,-32.316964
6,emp,cytosol,FBP == DHAP + GAP,FBA,9,NaN,NaN,14.8765,23.5555,NaN,-32.316964,-32.931868,-32.316964
7,emp,cytosol,DHAP == GAP,TPI,90,NaN,NaN,398.4248,307.9967,NaN,67.466280,66.850233,67.466280
8,emp,cytosol,GAP == G3P + ATP + NADH,GAPD,80,NaN,NaN,79.8354,0.01,NaN,61.835519,61.189792,61.835519
9,emp,cytosol,G3P == PEP,ENO,74,NaN,NaN,1.00E+06,999900,NaN,21.139391,19.390030,21.139391


### Run E-Flux2 for Oleic Acid 1

In [14]:
oleic_acid_medium = {
    'EX_ocdcea(e)': np.inf,
    'EX_h2o(e)': 1000.0,
    'EX_h(e)': 1000.0,
    'EX_nh4(e)': 1000.0,
    'EX_o2(e)': 1000.0,
    'EX_pi(e)': 1000.0,
    'EX_so4(e)': 1000.0,
}

oleic_acid_1_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=oleic_acid_1_gene_transcripts, 
    medium=oleic_acid_medium, 
    carbon_uptake_reaction='EX_ocdcea(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
oleic_acid_1_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/oleic_acid_1_gsm_eflux2.csv', index=False)

display(oleic_acid_1_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-3041.050412
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,2194.068288
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],2190.428963
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],2190.428963
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,2099.879235
964,O2t,o2 transport diffusion,o2[e] <=> o2[c],1405.430375
444,EX_o2(e),O2 exchange,o2[e] <=>,-1405.430375
609,CYOOm,cytochrome c oxidase mitochondrial,4.0 focytc[m] + 6.0 h[m] + o2[m] --> 4.0 ficyt...,1049.939618
1016,O2tm,O2 transport diffusion,o2[c] <=> o2[m],1049.939618
846,NADH2_u6m,NADH dehydrogenase mitochondrial,h[m] + nadh[m] + q6[m] --> nad[m] + q6h2[m],1029.211362


### Run E-Flux2 for Oleic Acid 2

In [15]:
oleic_acid_medium = {
    'EX_ocdcea(e)': np.inf,
    'EX_h2o(e)': 1000.0,
    'EX_h(e)': 1000.0,
    'EX_nh4(e)': 1000.0,
    'EX_o2(e)': 1000.0,
    'EX_pi(e)': 1000.0,
    'EX_so4(e)': 1000.0,
}

oleic_acid_2_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=oleic_acid_2_gene_transcripts, 
    medium=oleic_acid_medium, 
    carbon_uptake_reaction='EX_ocdcea(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
oleic_acid_2_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/oleic_acid_2_gsm_eflux2.csv', index=False)

display(oleic_acid_2_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-3041.050412
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,2194.068288
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],2190.428963
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],2190.428963
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,2099.879235
964,O2t,o2 transport diffusion,o2[e] <=> o2[c],1405.430375
444,EX_o2(e),O2 exchange,o2[e] <=>,-1405.430375
609,CYOOm,cytochrome c oxidase mitochondrial,4.0 focytc[m] + 6.0 h[m] + o2[m] --> 4.0 ficyt...,1049.939618
1016,O2tm,O2 transport diffusion,o2[c] <=> o2[m],1049.939618
846,NADH2_u6m,NADH dehydrogenase mitochondrial,h[m] + nadh[m] + q6[m] --> nad[m] + q6h2[m],1029.211362


### Run E-Flux2 for Oleic Acid 3

In [16]:
oleic_acid_medium = {
    'EX_ocdcea(e)': np.inf,
    'EX_h2o(e)': 1000.0,
    'EX_h(e)': 1000.0,
    'EX_nh4(e)': 1000.0,
    'EX_o2(e)': 1000.0,
    'EX_pi(e)': 1000.0,
    'EX_so4(e)': 1000.0,
}

oleic_acid_3_gsm_eflux2_df = eflux2(
    model=model, 
    gene_transcript_dictionary=oleic_acid_3_gene_transcripts, 
    medium=oleic_acid_medium, 
    carbon_uptake_reaction='EX_ocdcea(e)', 
    carbon_uptake_flux=100
)

# save the dataframe to a csv file
oleic_acid_3_gsm_eflux2_df.to_csv('../results/eflux2_gsm_fluxes/oleic_acid_3_gsm_eflux2.csv', index=False)

display(oleic_acid_3_gsm_eflux2_df.head(30))

,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-3041.050412
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,2194.068288
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],2190.428963
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],2190.428963
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,2099.879235
964,O2t,o2 transport diffusion,o2[e] <=> o2[c],1405.430375
444,EX_o2(e),O2 exchange,o2[e] <=>,-1405.430375
609,CYOOm,cytochrome c oxidase mitochondrial,4.0 focytc[m] + 6.0 h[m] + o2[m] --> 4.0 ficyt...,1049.939618
1016,O2tm,O2 transport diffusion,o2[c] <=> o2[m],1049.939618
846,NADH2_u6m,NADH dehydrogenase mitochondrial,h[m] + nadh[m] + q6[m] --> nad[m] + q6h2[m],1029.211362


### Oleic Acid Central Metabolism E-Flux2

In [17]:
# load oleic acid 13C MFA data
oleic_acid_13c_mfa_df = pd.read_csv('../data/13c_mfa/oleic_acid_13c_mfa.csv')

# only keep the reactions that are mapped to the GSM
oleic_acid_13c_mfa_df.dropna(subset = ["reaction_ids"], inplace=True)

# add the GSM flux predictions to the 13C-MFA dataframe
oleic_acid_central_eflux2_df = add_column_to_13c_flux_df(oleic_acid_13c_mfa_df, oleic_acid_1_gsm_eflux2_df, 'Oleic Acid 1 Flux')
oleic_acid_central_eflux2_df = add_column_to_13c_flux_df(oleic_acid_central_eflux2_df, oleic_acid_2_gsm_eflux2_df, 'Oleic Acid 2 Flux')
oleic_acid_central_eflux2_df = add_column_to_13c_flux_df(oleic_acid_central_eflux2_df, oleic_acid_3_gsm_eflux2_df, 'Oleic Acid 3 Flux')

# save the central fluxes as .csv
oleic_acid_central_eflux2_df.to_csv('../results/eflux2_central_fluxes/oleic_acid_central_eflux2.csv', index=False)

oleic_acid_central_eflux2_df

,pathway,compartment,13C_mfa_reaction,reaction_ids,13C_mfa_flux,13C_mfa_flux_95%_lower_bound,13C_mfa_flux_95%_upper_bound,Forward Flux,Backward Flux,Stdev,Oleic Acid 1 Flux,Oleic Acid 2 Flux,Oleic Acid 3 Flux
0,substrate_uptake,cytosol,Glucose(substrate) + ATP == G6P,GLCt1,0.0,NaN,NaN,[],[],NaN,0.000000e+00,0.000000e+00,0.000000e+00
1,substrate_uptake,cytosol,Glycerol(substrate) == GLYC,reverse_GLYCt,0.0,NaN,NaN,[],[],NaN,0.000000e+00,0.000000e+00,0.000000e+00
2,substrate_uptake,cytosol,Acetate(substrate) == AC,ACt2r,0.0,NaN,NaN,[],[],NaN,-1.009172e-15,2.896697e-15,-2.673874e-16
3,substrate_uptake,cytosol,oleic_acid(substrate) == AC,OCDCEAt,100.0,NaN,NaN,[],[],NaN,1.000000e+02,1.000000e+02,1.000000e+02
4,emp,cytosol,G6P == F6P,PGI,-77.4,NaN,NaN,70.1415,147.3525,NaN,-3.008972e+01,-3.008972e+01,-3.008972e+01
5,emp,cytosol,F6P + ATP == FBP,PFK or reverse_FBP,-65.7,NaN,NaN,7.80E+04,7.81E+04,NaN,-6.805403e+01,-6.805403e+01,-6.805403e+01
6,emp,cytosol,FBP == DHAP + GAP,FBA,-65.7,NaN,NaN,252.2214,318.2931,NaN,-6.805403e+01,-6.805403e+01,-6.805403e+01
7,emp,cytosol,DHAP == GAP,TPI,-65.7,NaN,NaN,0,66.0717,NaN,-7.090979e+01,-7.090979e+01,-7.090979e+01
8,emp,cytosol,GAP == G3P + ATP + NADH,GAPD,-130.5,NaN,NaN,337.113,467.5986,NaN,-1.437074e+02,-1.437074e+02,-1.437074e+02
9,emp,cytosol,G3P == PEP,ENO,-168.3,NaN,NaN,1110.7242,1279.2996,NaN,-1.437074e+02,-1.437074e+02,-1.437074e+02
